In [18]:
import sounddevice
import numpy
import time
import yaml

ModuleNotFoundError: No module named 'sounddevice'

In [ ]:
phonon_mesh_filepath = './data/BaS_Fm3m/mesh.yaml'
sample_rate = 44100
timelength = 5  # length of sample in seconds
min_audible = 20# minimum audible frequency in herz
max_audible = 20E3 # maximum audible frequency in herz
min_phonon = None # minimum phonon frequency in THz, if not set it is extracted from the phonon calc
max_phonon = None # maximum phonon frequency in THz, if not set it is extracted from the phonon calc

In [ ]:
def callback(outdata: numpy.ndarray, frames: int, time, status, audible_frequencies, sample_rate) -> None:
    """writes sound output to 'outdata' from sound_queue."""
    result = None

    for frequency in audible_frequencies:
        t = (start_index + numpy.arange(frames)) / sample_rate
        t = t.reshape(-1, 1)

        wave = numpy.sin(2 * numpy.pi * frequency * t)

        if result is None:
            result = wave
        else:
            result += wave

        frequencies[frequency] += frames

    if result is None:
        result = numpy.arange(frames) / SAMPLE_RATE
        result = result.reshape(-1, 1)

    outdata[:] = result
    
def phonon_to_audible(phonon_frequencies):
    """linearly maps phonon frequencies (in THz) to frequencies in the audible range (returns in Hz)"""
    
    if min_phonon is None:
        min_phonon = min(phonon_frequencies)
    min_phonon_hz = min_phonon*1E12
    
    if max_phonon is None:
        max_phonon = max(phonon_frequencies)
    max_phonon_hz = max_phonon*1E12
    
    phonon_frequencies_hz = np.array(phonon_frequencies)*1E12
    
    scale_factor = (max_audible - min_audible) / (max_phonon_hz - min_phonon_hz)
    
    audible_frequencies = [ scale_factor*(frequency-min_phonon_hz) + min_audible for frequency in phonon_frequencies_hz]
    
    return audible_frequencies

In [2]:
# read yaml data 
with open(phonon_mesh_filepath) as f:
    data = yaml.safe_load(f)

# extract list of unique frequencies - these are in THz
phonon_frequencies = list(set([dictionary['frequency'] for dictionary in data['phonon'][0]['band']]))
# remove any imaginary modes
phonon_frequencies = [frequency for frequency in phonon_frequencies if frequency > 0]

# TODO: convert frequencies to herz within audible range
audible_frequencies = phonon_to_audible(phonon_frequencies)

# create output stream
stream = sounddevice.OutputStream(channels=2, blocksize=SAMPLE_RATE, 
            samplerate=SAMPLE_RATE, callback=callback, audible_frequencies, sample_rate)

# start stream and keep running for timelength
stream.start()
time.sleep(timelength)

In [6]:
# specify Materials Project ID or provide your own mesh.yaml or band.yaml
# have in gamma-point mode or band dispersion mode
# gamma point mode: play chord, temperature slider, keyboard of notes
# band dispersion mode: play as moving through reciprocal space, or select position to play chord

# host as a web app
# use as an education outreach tool
# use for undergraduate education
# use for musical performance?
# learning points: role of defects